In [1]:
#Below is the code to generate the estimation circuit from the BCS Trotter circuit 
#as well as their randomized compiling version


from qiskit import IBMQ, Aer
from qiskit import QuantumCircuit, transpile, assemble
import numpy as np
from functions_randomized_compiling import *
from functions_estimation_circuit import *

#Which time steps of Trotter circuit do you want to build the corresponding estimation circuit
initial_step=1 # initial Trotter step 
final_step=15 #final Trotter step


#choose which randomized compiling version you want for the estimation circuit
randomized_compiling='crosstalk'#twirl cross-talk
#randomized_compiling='standard'  #standard randomized compiling, only on active qubits of the CNOT
nb_twirls=30 #number of twirl

backend = 'ibm_lagos'
coupling_map=[[0, 1],[1, 0],[1, 2],[1, 3],[2, 1],[3, 1],[3, 5],[4, 5],[5, 3],[5, 4],[5, 6],[6, 5]]
   

In [3]:
#Building the estimation circuit
dt=0.2 #Trotter time step
g=0.5 #coupling constant

qc_estimation=list(list([] for _ in range(nb_twirls)) for _ in range(final_step-initial_step+1))
for t in range(initial_step,final_step+1):
    T=dt*t
    #upload the BCS Trotter_circuit
    qc=QuantumCircuit.from_qasm_file('data/ZZZ/T='+str(round(T,1))+'/trotter_g='+str(g)+'_dt='+str(round(dt,3))+'_backend='+str(backend)+'.qasm')
    qc2=preserve_cnot(qc)# keep only the CNOT of the circuit
    for k in range(nb_twirls):
        qc3=add_random(qc2)
        if randomized_compiling=='crosstalk':
            qc_estimation[t-initial_step][k]=crosstalk_twirl(qc3,coupling_map)
        elif randomized_compiling=='standard':
            qc_estimation[t-initial_step][k]=standard_twirl(qc3)
            

Estimation circuits are only stored in the folder ZZZ measurements, there are the same estimation circuit for both measurement because we removed 1-qubit gate operations.
300 standard randomized compiling estimation circuits circuits from initial_step=1 to final_step=15 are available in QASM format in 'data/ZZZ/t="time_step"/estimation_circuit/Quantum_Circuit/estimation_standard_twirl_g=0.5=dt=0.2_backend_ibm_lagos_"twirl number".qasm'
300 (for T<1.8) and 600 (for T>=1.8) crosstalk twirled  estimation circuits from initial_step=1 to final_step=15 are available in QASM format in 'data/ZZZ/t="time step"/estimation_circuit/Quantum_Circuit/estimation_crosstalk_twirl_g=0.5=dt=0.2_backend_ibm_lagos_"twirl number".qasm'

300 standard twirl and crosstalk twirl circuits for backend='ibmq_ehningen' and measurement='XYZ'

In [4]:
#Simulation of the generated circuits
#measured qubits should be always in state '0' without any noisy gates

#Simulation of the randomized compiling  estimation circuit on a perfect QC

nb_shots=32000 #number of shots
sim = Aer.get_backend('aer_simulator')
counts=list(list({} for _ in range(nb_twirls)) for _ in range(final_step-initial_step+1))
for t in range(initial_step,final_step+1):
    for i in range(nb_twirls):
        qobj = assemble(qc_estimation[t-initial_step][i])
        counts[t-initial_step][i]= sim.run(qobj,shots=nb_shots).result().get_counts()


In [5]:
print(counts)

[[{'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}], [{'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}, {'000': 32000}], [{'000': 32000}, {'000': 32000}, {'